In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import ast
import os
import xml.etree.ElementTree
from collections import Counter, defaultdict
import pandas as pd
from scipy import stats
import random

## Load Data

In [ ]:
# qid docid docno doclen rel positions
def loadData():
    qid2reldoc = defaultdict(list)
    qid2irrdoc = defaultdict(list)
    docid2doclen = defaultdict(int)
    docid2flatlist = defaultdict(list)
    
    i = 0 
    with open("wtall_qrels_pos.clean.txt") as f:
        for l in f:
            # split row
            fields = l.split("\t")
            
            # get the data
            qid = int(fields[0])
            docid = int(fields[1])
            doclen=int(fields[3])
            rel=int(fields[4])
            flat_list = [item for sublist in ast.literal_eval(fields[5]) for item in sublist]

            
            # put data in dicts
            docid2doclen[docid] = doclen
            docid2flatlist[qid, docid] = flat_list
            if rel > 0:
                qid2reldoc[qid].append(docid)
            else:
                qid2irrdoc[qid].append(docid)

    return  qid2reldoc, qid2irrdoc, docid2doclen, docid2flatlist

In [ ]:
qid2reldoc, qid2irrdoc, docid2doclen, docid2flatlist = loadData()

In [ ]:
print qid2irrdoc[1]
print qid2reldoc[1]

a = set(qid2reldoc[1])
b = set(qid2irrdoc[1])
a.intersection(b)
print len(b)
print len(a)
print len(a.intersection(b))

In [ ]:
def loadQidInfo():
    qidDict = dict()
    for filename in os.listdir(os.curdir):
        if filename.endswith(".xml"):
            e = xml.etree.ElementTree.parse(filename).getroot()
            for query in e.findall('topic'):
                qidDict[int(query.get("number"))] = tuple([int(query.get("number")), query.get("type"), query[0].text])
    return qidDict

In [ ]:
qidInfo = loadQidInfo()

## 1. Stats on document length

In [ ]:
# ALL DOCs with RELEVANCE
def plotStats(docid2doclen):
    sorted_per_freq = Counter(docid2doclen.values()).most_common()
    sorted_per_length = sorted(sorted_per_freq, key=lambda i: i[0])
    doc_len, doc_len_freq = zip(*sorted_per_length)
    plt.loglog(doc_len, doc_len_freq, 'r.')
    plt.xlabel("DocLen")
    plt.ylabel("Freq")
    
    
plotStats(docid2doclen)
stats.describe(docid2doclen.values())

In [ ]:
# ONLY RELEVANT DOCS
relevantdocs = [item for sublist in qid2reldoc.values() for item in sublist]
print len(relevantdocs)
print Counter(relevantdocs).most_common(20)
relevant_doclen_dict = {docid:docid2doclen[docid] for docid in relevantdocs}

print "---------------------------"
print "Num Relevant Docs (unique): ", len(relevant_doclen_dict)
print "Stats: ",stats.describe(relevant_doclen_dict.values())
print "Median: ", np.median(relevant_doclen_dict.values())
plotStats(relevant_doclen_dict)

## 2. Build OCCURENCE Probability MATRIX

In [ ]:
# Samples irrelevant from the same query ID

np.random.seed(6)
def sample_irr_for_qidA(qid, qid2reldoc, qid2irrdoc):
    num_irr = len(qid2reldoc[qid])
    # sample larger than population
    if num_irr >= len(qid2irrdoc[qid]): 
        irr_docids = qid2irrdoc[qid]
    else:
        irr_docids = random.sample(qid2irrdoc[qid],num_irr)
    return irr_docids

In [ ]:
def computeProbabDistribution(flatlists, max_doc_len):
    occurence_ndarray = np.zeros(max_doc_len)
    for flatlist in flatlists:
        filtered_flatlist = [x for x in flatlist if x < max_doc_len]
        occurence_ndarray[filtered_flatlist] += 1
    occurence_ndarray = occurence_ndarray/len(flatlists)
    return occurence_ndarray

In [ ]:
qid_RelevantDocsDistrib = dict()

# A. only irrelevant for that query
qid_IrrelevantDocDistribA = dict() 

# ALL
qid_allDocsDistrib = dict()

In [ ]:
# at PER QUERY level

#### RELEVANT
for qid, relDocidList in qid2reldoc.iteritems():
    flatlists = [docid2flatlist[qid, docid] for docid in relDocidList]
    occ_probab_distrib = computeProbabDistribution(flatlists, 2000)
    if True in np.isnan(occ_probab_distrib):
        occ_probab_distrib = np.zeros(2000)
    qid_RelevantDocsDistrib[qid] = occ_probab_distrib
    
print len(qid_RelevantDocsDistrib)
print qid_RelevantDocsDistrib.keys()
print qid_RelevantDocsDistrib[2]

In [ ]:
#### IRELEVANT only irrelevant for that query
for qid, irelDocidList in qid2irrdoc.iteritems():
    if qid in qid_RelevantDocsDistrib.keys():
        #irelDocidList = sample_irr_for_qidA(qid, qid2reldoc, qid2irrdoc)
        flatlists = [docid2flatlist[qid, docid] for docid in irelDocidList]
        occ_probab_distrib = computeProbabDistribution(flatlists, 2000)
        qid_IrrelevantDocDistribA[qid] = occ_probab_distrib

print len(qid_IrrelevantDocDistribA)
print qid_IrrelevantDocDistribA.keys()

In [ ]:
#### ALL
for qid, irelDocidList in qid2irrdoc.iteritems():
    if qid in qid_RelevantDocsDistrib.keys():
        all_docids = list()
        all_docids.extend(irelDocidList)
        all_docids.extend(qid2reldoc[qid])

        flatlists = [docid2flatlist[qid, docid] for docid in all_docids if (qid, docid) in docid2flatlist]
        occ_probab_distrib = computeProbabDistribution(flatlists, 2000)
        qid_allDocsDistrib[qid] = occ_probab_distrib

print len(qid_allDocsDistrib)
print qid_allDocsDistrib.keys()

In [ ]:
def bucketandreshape(x, bucket_size):
    return x.reshape(x.size // bucket_size, bucket_size).sum(axis=1)

## 3.1 KL divergence

https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.entropy.html

scipy.stats.entropy(pk, qk=None, base=None)

Calculate the entropy of a distribution for given probability values.
If only probabilities pk are given, the entropy is calculated as S = -sum(pk * log(pk), axis=0).
If qk is not None, then compute the Kullback-Leibler divergence S = sum(pk * log(pk / qk), axis=0).

This routine will normalize pk and qk if they don’t sum to 1.

Parameters:	

    pk : sequence
       Defines the (discrete) distribution. pk[i] is the (possibly unnormalized) probability of event i.
       
    qk : sequence, optional
        Sequence against which the relative entropy is computed. Should be in the same format as pk.
        
    base : float, optional
        The logarithmic base to use, defaults to e (natural logarithm).
Returns:	

    S : float
    The calculated entropy.


In [ ]:
from sklearn import metrics

bucket_size = 100

kl_Rel_IrrelA = dict()
kl_Rel_All = dict()
mutual_info = dict() # this in sklearn is done on labels, not prob_distrib
for qid in qid_RelevantDocsDistrib.keys():
    print 
    print qid
    # no bucketing
    a = qid_RelevantDocsDistrib[qid]
    b = qid_IrrelevantDocDistribA[qid]
    c = qid_allDocsDistrib[qid]
    
    # with bucketing

    a = bucketandreshape(a, bucket_size)
    print a
    b = bucketandreshape(b, bucket_size)
    print b
    c = bucketandreshape(c, bucket_size)
    print c

    
    kl_Rel_IrrelA[qid] = stats.entropy(a, b, 2.0)
    kl_Rel_All[qid] = stats.entropy(a, c, 2.0)
    mutual_info[qid] = metrics.mutual_info_score(a, b)
    print kl_Rel_IrrelA[qid], kl_Rel_All[qid], mutual_info[qid]

sklearn.metrics.mutual_info_score([0,1],[1,0]) 
https://datascience.stackexchange.com/questions/9262/calculating-kl-divergence-in-python

Kullback-Leibler divergence is fragile, unfortunately. On above example it is not well-defined: KL([0,1],[1,0]) causes a division by zero, and tends to infinity. It is also asymmetric.

https://en.wikipedia.org/wiki/Mutual_information

https://math.stackexchange.com/questions/21190/justification-for-infinite-kl-divergence
https://en.wikipedia.org/wiki/Talk%3AKullback%E2%80%93Leibler_divergence
https://en.wikipedia.org/wiki/Statistical_distance

## 3.2 Jaccard coef

needs to be binary

In [ ]:
jack_Rel_Irrel = dict()
jack_Rel_All = dict()
bucket_size = 100

for qid in qid_RelevantDocsDistrib.keys():
    print 
    print qid
    # no bucketing
    a = qid_RelevantDocsDistrib[qid]
    b = qid_IrrelevantDocDistribA[qid]
    c = qid_allDocsDistrib[qid]
    
    # with bucketing

    a1 = bucketandreshape(a, bucket_size)
    a = [0 if i<a1.mean() else 1 for i in a1]
    print a
    b1 = bucketandreshape(b, bucket_size)
    b = [0 if i<b1.mean() else 1 for i in b1]
    print b
    c1 = bucketandreshape(c, bucket_size)
    c = [0 if i<c1.mean() else 1 for i in c1]
    print c
    
    # get the coef!
    jack_Rel_Irrel[qid] = metrics.jaccard_similarity_score(a,b)
    jack_Rel_All[qid] = metrics.jaccard_similarity_score(a,c)
    
    print jack_Rel_Irrel[qid], jack_Rel_All[qid]

#### Lots of other metrics: http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics
#### Other measures for statistical difference between 2 probab distrib: https://en.wikipedia.org/wiki/Statistical_distance

## 4. PLOTS with different granularity

### all 2000 positions

In [ ]:
for qid in qid_RelevantDocsDistrib.keys():
    plt.figure(figsize=(16, 3))
    
    plt.subplot(1,3,1)
    #if qid in qid_RelevantDocsDistribFlat:
    a = qid_RelevantDocsDistrib[qid]
    a = np.insert(a, 0, np.zeros(40))
    plt.imshow(a.reshape((51, 40)), cmap="hot") #, interpolation='sinc')
    
    plt.subplot(1,3,2)
    b = qid_IrrelevantDocDistribA[qid]
    b = np.insert(b, 0, np.zeros(40))
    plt.imshow(b.reshape((51, 40)), cmap="hot") #, interpolation='sinc')
    
    plt.subplot(1,3,3)
    
    c = qid_allDocsDistrib[qid]
    c = np.insert(c, 0, np.zeros(40))
    plt.imshow(c.reshape((51, 40)), cmap="hot") #, interpolation='sinc')
    

    plt.suptitle(str(qidInfo[qid])+ " " + str(kl_Rel_IrrelA[qid]) + " " + str(kl_Rel_All[qid]))

    plt.show()
    

### aggregated bucket size 20 => 100 positions

In [ ]:
qid_label = []
X = []
X_irrel = []
bucket_size = 20

for qid in qid_RelevantDocsDistrib.keys():
    plt.figure(figsize=(16, 3))
    
    qid_label.append(qid)
    
    plt.subplot(1,3,1)
    a = qid_RelevantDocsDistrib[qid]
    a = bucketandreshape(a, bucket_size)
    X.append(a)
    plt.imshow(a.reshape((10, 10)), cmap="hot") #, interpolation='sinc')
    
    plt.subplot(1,3,2)
    b = qid_IrrelevantDocDistribA[qid]
    b = bucketandreshape(b, bucket_size)
    X_irrel.append(b)
    plt.imshow(b.reshape((10, 10)), cmap="hot") #, interpolation='sinc')
    
    plt.subplot(1,3,3)
    c = qid_allDocsDistrib[qid]
    c = bucketandreshape(c, bucket_size)
    plt.imshow(c.reshape((10, 10)), cmap="hot") #, interpolation='sinc')
    

    plt.suptitle(str(qidInfo[qid])+ " " + str(jack_Rel_Irrel[qid]) + " " + str(jack_Rel_All[qid]))

    plt.show()
    
X = np.array(X)
X_irrel = np.array(X_irrel)

### Plot 5 random qid , with dimension 100

In [ ]:
np.random.seed(6)

qid_sample = random.sample(qid2reldoc.keys(),5)
plt.figure(figsize=(16, 8))
for qid in qid_sample:
    a = bucketandreshape(qid_RelevantDocsDistrib[qid], 20)
    plt.plot(a, label=qid)
plt.legend()
plt.show()

## 5. Pairwise distances

http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise_distances.html#sklearn.metrics.pairwise_distances

Valid values for metric are:
From scikit-learn: [‘cityblock’, ‘cosine’, ‘euclidean’, ‘l1’, ‘l2’, ‘manhattan’]. These metrics support sparse matrix inputs.
From scipy.spatial.distance: [‘braycurtis’, ‘canberra’, ‘chebyshev’, ‘correlation’, ‘dice’, ‘hamming’, ‘jaccard’, ‘kulsinski’, ‘mahalanobis’, ‘matching’, ‘minkowski’, ‘rogerstanimoto’, ‘russellrao’, ‘seuclidean’, ‘sokalmichener’, ‘sokalsneath’, ‘sqeuclidean’, ‘yule’] See the documentation for scipy.spatial.distance for details on these metrics. These metrics do not support sparse matrix inputs.

In [ ]:
from sklearn import metrics

print len(X), len(X[1])
print len(X_irrel), len(X_irrel[1])
new_X = np.concatenate([X, X_irrel], axis = 1)

for metric in ['euclidean','cosine']:
    D = metrics.pairwise_distances(X,  metric=metric) 
    print metric
    print D
    print "------------------------"
    plt.figure(figsize=(10, 10))
    plt.imshow(D) # interpolation='nearest', cmap=plt.cm.gnuplot2, vmin=0
    plt.title(metric)

In [ ]:
# the bigger the value the more different they are

from sklearn import metrics

print len(X), len(X[1])
print len(X_irrel), len(X_irrel[1])
print len(qid_label)

for metric in ['euclidean','cosine']:
    D = metrics.pairwise_distances(X, X_irrel,  metric=metric) 
    print metric
#     print D
    print "------------------------"
    plt.figure(figsize=(10, 10))
    plt.imshow(D) # interpolation='nearest', cmap=plt.cm.gnuplot2, vmin=0
    plt.title(metric)
    
    for i in range(D.shape[0]):
        print D[i,i], qidInfo[qid_label[i]]
        
#     print D[135,135] , qidInfo[qid_label[135]]

In [ ]:
from sklearn import metrics

print len(X), len(X[1])

for metric in ['euclidean', 'cosine']:
    D = metrics.pairwise_distances(X,  metric=metric) 
    print metric
    print D
    print "------------------------"
    plt.figure(figsize=(10, 10))
    plt.imshow(D) # interpolation='nearest', cmap=plt.cm.gnuplot2, vmin=0
    plt.title(metric)

In [ ]:
D.copy()
# D[5,98]
# D[35, 108]

In [ ]:
from scipy.sparse.csgraph import connected_components

thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for t in thresholds:
    graph = D.copy()
    graph = 1 - graph
    graph[graph<t] = 0.0
    print graph.shape, "threshold: ", t
    n_comp, labels = connected_components(graph, directed=False, connection='weak', return_labels=True)
    print n_comp
#     print labels
    print Counter(labels).most_common()
    print 

In [ ]:
### EQUIVALENT TO prev cell

import networkx as nx

thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

for t in thresholds[6:10]:
    graph = D.copy()
    graph = 1 - graph # now we have similarity instead of distance
    graph[graph<t] = 0.0
    
    print
    print
    print graph.sum()
    print graph.shape, "threshold: ", t

    G = nx.from_numpy_matrix(graph)
    cc = nx.connected_components(G)
#     cc = list(nx.connected_component_subgraphs(G))
    print "Number of connected components", nx.number_connected_components(G)

    for g in cc:
        print g

    
# # https://networkx.github.io/documentation/networkx-1.9.1/reference/algorithms.component.html


# nx.draw_random(g)
# nx.draw_circular(g)
# nx.draw_spectral(g)